**Import Some Libraries**

In [124]:
import math as m
import numpy as np
from tabulate import tabulate

**Function Definitions**

In [125]:
def sqrt(number):
    return m.sqrt(number)

def find_a(fc,fy,As,b):
    a = (As*fy)/(0.85*fc*b)
    return a

def find_beta1(fc):
    if fc <= 4:
        beta1 = 0.85
    elif fc >= 8:
        beta1 = 0.65
    else:
        beta1 = 0.85 - .05*(fc-4)
    return beta1

def find_c(a, beta1):
    c = a/beta1
    return c

def find_Mn(fc,fy,As,b,d):
    a = find_a(fc,fy,As,b)
    Mn = As*fy*(d-a/2)
    return Mn

def find_rho(As,b,d):
    rho = As/(b*d)
    return rho

def find_ey(fy,Es):
    if fy == 60:
        ey = 0.002
    else:
        ey = fy/Es
    return ey 

def find_rho_max(fc,fy,beta1,ey):
    rho_max = 0.85*fc/(fy*beta1)*(.003)/(.003+ey+.003)
    return rho_max

def find_rho_bal(fc,fy,beta1,ey):
    rho_bal = 0.85*fc/(fy*beta1)*(.003)/(.003+ey)
    return rho_bal

def find_rho_min(fc,fy):
    rho_min = 3*sqrt(fc*1000)/fy/1000
    if rho_min > 200/(fy*1000):
        rho_min = 200/(fy*1000)
    return rho_min

def find_phi(rho,rho_max,rho_bal):
    if rho < rho_max and rho <= rho_bal:
        phi = 0.9
    elif rho_max <= rho <= rho_bal:
        phi = 0.6500000001
    else:
        phi = 0.65
    return phi

def comment_on_rho(rho,rho_min,rho_max):
    if rho < rho_min:
        print("You need to add more rebar!")
        print()
    elif rho > rho_max:
        print("You should remove some rebar or increase your beam or slab depth!")
        print()
    else:
        print("Your reinforcement ratio is good.")
        print()

def comment_on_phi(phi):
    if phi == 0.9:
        print(f"You're in the tension-controlled zone, which is good, so phi = {phi:.2f} (for flexure).")
        print()
    elif 0.65 < phi < 0.9:
        print(f"Hmmm, you're in the transition zone... Check your design. phi = {phi:.2f} \n (assuming phi for compression-controlled flexure).")
        print()
    else:
        print(f"Yikes! You're over-reinforced! Sudden failures are possible. \n Actually... please fix it. Don't do it. \n Please! Babies might die! phi = {phi:.2f} (Bad phi for flexure)")
        print()

def comment_on_h(d,h):
    if h < d:
        print("Check your dimensions, d cannot be larger than h! We're not in the Twilight Zone...")
        print()
    if h < d + 1.5:
        print("You might want to increase h to ensure proper concrete cover.")
        print()

    
def find_phi_Mn(phi,Mn):
    phi_Mn = phi*Mn
    return phi_Mn

def find_Av_min(fc,fy,b): # ACI 318-19 Table 9.6.3.4 for nonprestressed members
    Av_min = 0.0
    Av_min = .75*b/sqrt(fc*1000)/fy/1000
    if Av_min < 50*b/fy/1000:
        Av_min = 50*b/fy/1000   
    return Av_min

def find_lambda_size(d): # ACI 318-19 22.5.5.1.3
    if d <= 10:
        lambda_size = 1
    else:
        lambda_size = sqrt(2/(1+d/10))
    return lambda_size

def find_Vc(fc,b,d,h,Av,Av_min,member_type,lambda_size,lambda_conc): # ACI 318-19 7.6.3.1 or 9.6.3 and 22.5.5 (assuming no axial load)
    Vc = 0.0
    if member_type == "slab":
        if Av >= Av_min:
            print("Slab with Av >= Av_min (We have enough stirrups).")
            # 22.5.5.1 (a) chosen if Av >= Av_min
            Vc = 2*lambda_conc*sqrt(fc*1000)*b*d/1000                                     
            if Vc < 8*lambda_conc*(Av/b/d)**(1/3)*m.sqrt(fc*1000)*b*d/1000:
                # 22.5.5.1 (b) chosen if it is larger than (a)
                Vc = 8*lambda_conc*(Av/b/d)**(1/3)*m.sqrt(fc*1000)*b*d/1000                
        else:
            print("This is a slab with Av < Av_min (No stirrups or not enough stirrups).")
            # 22.5.5.1 (c) chosen if Av < Av_min                
            Vc = 8*lambda_size*lambda_conc*(Av_min/b/d)**(1/3)*m.sqrt(fc*1000)*b*d/1000   
    
    elif member_type == "beam":
        if Av >= Av_min:
            print("Beam with Av >= Av_min (We have enough stirrups).")
            Vc = 2*lambda_conc*sqrt(fc*1000)*b*d/1000                                     # 22.5.5.1 (a)
            if Vc < 8*lambda_conc*(Av/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000:
                Vc = 8*lambda_conc*(Av/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000                # 22.5.5.1 (b)
        else: # Av < Av_min
            print("This is a beam with Av < Av_min (No stirrups or not enough stirrups).")
            if h <= 10:
                print("... but h <= 10 inches, so we don't have meet Av_min requirements per Table 9.6.3.1 to be able to use 22.5.5.1 (c).")    
                # use 22.5.5.1 (c) assuming rectangular beam with Av < Av_min and h <= 10 inches per Table 9.6.3.1                
                Vc = 8*lambda_size*lambda_conc*(Av_min/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000
            else: # h > 10
                print("... and h > 10 inches, so we have to meet Av_min requirements unless we have enough Vc for Vu at critical section per 9.6.3 AND 22.5.5.1 (c).")
                Vc = 8*lambda_size*lambda_conc*(Av_min/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000
                # 9.6.3.1 limits shear strength for beams deeper than 10 inches
                if Vc > lambda_conc*sqrt(fc*1000)*b*d/1000:
                    Vc = lambda_conc*sqrt(fc*1000)*b*d/1000
                    print("Might need some stirrups... 9.6.3.1 is limiting our shear strength!")
    print()
    return Vc

def check_max_Vc(Vc,fc,b,d,lambda_conc): # ACI 318-19
    if Vc > 5*lambda_conc*sqrt(fc*1000)*b*d/1000:  # ACI 318-19 22.5.5.1.1
        Vc = 5*lambda_conc*sqrt(fc*1000)*b*d/1000
        print("Vc is being capped by 22.5.5.1.1... beam or slab might be have too small of depth (h)!")
        print()
    return Vc  

def adjust_Vc_with_Nu(Vc,fc,b,d,h,Nu): # ACI 318-19 22.5.5.1 and 22.5.5.1.2
    if Nu == 0:
        print("No axial load. No adjustment to Vc needed using Nu.")
        print()
    elif Nu != 0 and Nu/(6*b*h) <= .05*fc:
        Vc = Vc + Nu/6/(b*h)*b*d
        print("Axial load is present. Adjusting Vc using Nu, of which is lower than 0.05fc (See 22.5.5.1.2).")
        print()
    else:
        Vc = Vc + .05*fc*b*d
        print("Axial load is present. Adjusting Vc using 22.5.5.1.2 since Nu is too high.")
        print()
        
    return Vc

In [126]:
# Commonly not changed values
Es = 29000                  # ksi

#Common inputs to change
fy = 60                     # ksi
fc = 5                      # ksi
beta1 = find_beta1(fc)      # from ACI 318-19 Table
b = 12                      # in
h = 12                      # in
d = 10                      # in
As = 3                     # in^2
Av = 0                      # in^2
member_type = "beam"        # beam or slab
lambda_conc = 1             # 1 for normal weight, 1.2 for lightweight
Nu = 208                     # kips, positive for compression, negative for tension

print("___GIVEN___")
print(f"fy = {fy} ksi")
print(f"Es = {Es} ksi")
print(f"fc = {fc} ksi")
print(f"b = {b} in")
print(f"h = {h} in")
print(f"d = {d} in")
print(f"As = {As} in^2")
print(f"Av = {Av} in^2")
print(f"member type = {member_type}")
print(f"lambda_conc = {lambda_conc}")
print(f"Nu = {Nu} kips")
print()

#Find values
lambda_size = find_lambda_size(d)
Av_min = find_Av_min(fc,fy,b)
a = find_a(fc,fy,As,b)
c = find_c(a, beta1)
ey = find_ey(fy,Es)

rho = find_rho(As,b,d)
rho_min = find_rho_min(fc,fy)
rho_max = find_rho_max(fc,fy,beta1,ey)
rho_bal = find_rho_bal(fc,fy,beta1,ey)

Mn = find_Mn(fc,fy,As,b,d)
phi = find_phi(rho,rho_max,rho_bal)
phi_Mn = find_phi_Mn(phi,Mn)


print("___NOTES AND CHECKS___")
# Shear, Vc, is now wildly complicated to calculate, so checks are done during calculations for now, 
# which is why they are in this "checks" section.
Vc = find_Vc(fc,b,d,h,Av,Av_min,member_type,lambda_size,lambda_conc)
Vc = adjust_Vc_with_Nu(Vc,fc,b,d,h,Nu)
Vc = check_max_Vc(Vc,fc,b,d,lambda_conc)
phi_Vc = 0.75*Vc
comment_on_h(d,h)
comment_on_rho(rho,rho_min,rho_max)
comment_on_phi(phi)

print("___CALCULATION TABLE___")

value_table_with_formulas = [
    ["fy (ksi)", fy, "Given"],
    ["Es (ksi)", Es, "Given"],
    ["f'c (ksi)", fc, "Given"],
    ["b (in)", b, "Given"],
    ["d (in)", d, "Given"],
    ["As (in^2)", As, "Given"],
    ["Av (in^2)", Av, "Given"],
    [],
    ["Av_min (in^2)", f"{Av_min:.3f}", "ACI 318-19 Table 9.6.3.4 for nonprestressed members"],
    ["lambda_size", f"{lambda_size:.3f}", "ACI 318-19 22.5.5.1.3"],
    ["ey", f"{ey:.5f}", "ey = fy/Es or 0.002 if fy = 60 ksi"],
    ["beta1", f"{beta1:.2f}", "ACI 318-19 Table 22.2.2.4.3"],
    ["a (in)", f"{a:.2f}", "a = As*fy/(0.85*fc*b)"],
    ["c (in)", f"{c:.2f}", "c = a/beta1"],
    ["rho", f"{rho:.4f}", "rho = As/(b*d)"],
    ["rho_max", f"{rho_max:.4f}", "rho_max = 0.85*fc/(fy*beta1)*(.003)/(.003+ey+.003)"],
    ["rho_bal", f"{rho_bal:.4f}", "rho_bal = 0.85*fc/(fy*beta1)*(.003)/(.003+ey)"],
    ["rho_min", f"{rho_min:.4f}", "rho_min = 3*sqrt(fc*1000)/fy/1000 and less than 200/(fy*1000)"],
    [],
    ["phi", f"{phi:.2f}", "Based on rho, rho_bal, rho_max"],
    ["Mn (kip-in)", f"{Mn:.2f}", "Mn = As*fy*(d-a/2)"],
    ["Vc (kip)", f"{Vc:.2f}", "ACI 318-19 7.6.3.1 or 9.6.3 and 22.5.5 (assuming no axial load)"],
    [],
    ["phi_Mn (kip-in)", f"{phi_Mn:.2f}", "phi_Mn = phi*Mn"],
    ["phi_Mn (kip-ft)", f"{phi_Mn/12:.2f}", "phi_Mn in kip-ft"],
    ["phi_Vc (kip)", f"{phi_Vc:.2f}", "phi_Vc = 0.75*Vc"]
]

headers = ["Parameter", "Value", "Formula/Notes"]

print(tabulate(value_table_with_formulas, headers, tablefmt="simple", numalign="right"))

___GIVEN___
fy = 60 ksi
Es = 29000 ksi
fc = 5 ksi
b = 12 in
h = 12 in
d = 10 in
As = 3 in^2
Av = 0 in^2
member type = beam
lambda_conc = 1
Nu = 208 kips

___NOTES AND CHECKS___
This is a beam with Av < Av_min (No stirrups or not enough stirrups).
... and h > 10 inches, so we have to meet Av_min requirements unless we have enough Vc for Vu at critical section per 9.6.3 AND 22.5.5.1 (c).

Axial load is present. Adjusting Vc using Nu, of which is lower than 0.05fc (See 22.5.5.1.2).

Your reinforcement ratio is good.

You're in the tension-controlled zone, which is good, so phi = 0.90 (for flexure).

___CALCULATION TABLE___
Parameter          Value  Formula/Notes
---------------  -------  ---------------------------------------------------------------
fy (ksi)              60  Given
Es (ksi)           29000  Given
f'c (ksi)              5  Given
b (in)                12  Given
d (in)                10  Given
As (in^2)              3  Given
Av (in^2)              0  Given

Av_min (in^2)    